<a href="https://www.kaggle.com/code/elcientfico/driving-a-taxi-using-tree-methods-and-convolutions?scriptVersionId=148177828" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [40]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow.keras.layers import Input,Conv2D,GlobalMaxPooling2D# Input data files are available in the read-only "../input/" directory
from tensorflow.keras.models import Model# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from PIL import Image
import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [41]:
im_size = Image.open("/kaggle/input/gta-san-andreas-driving-dataset-with-label/GTA_SanAndreasDriving/screenshots/1000_1010.png").size
im_size = (im_size[1],im_size[0],3)

In [42]:
class conv_model:
    def __init__(self):
        pass
def conv_model(ishape = (512,512,3), filters = 256, kernel_s = 3, strides = 3):
    x1  = Input(shape = ishape)
    if len(ishape)<3:
        x2 = tf.expand_dims(x1,-1)
        x2 = Conv2D(filters,kernel_s,strides = strides)(x2)
    else:
        x2 = Conv2D(filters,kernel_s,strides = strides)(x1)
    x3 = Conv2D(filters,kernel_s,strides = strides)(x2)
    x4 = GlobalMaxPooling2D()(x3)
    model = Model(x1,x4)
    return model


In [60]:
model = conv_model(im_size,1024,3,2)
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 600, 795, 3)]     0         
                                                                 
 conv2d_6 (Conv2D)           (None, 299, 397, 1024)    28672     
                                                                 
 conv2d_7 (Conv2D)           (None, 149, 198, 1024)    9438208   
                                                                 
 global_max_pooling2d_3 (Glo  (None, 1024)             0         
 balMaxPooling2D)                                                
                                                                 
Total params: 9,466,880
Trainable params: 9,466,880
Non-trainable params: 0
_________________________________________________________________


In [61]:
ss_path = "/kaggle/input/gta-san-andreas-driving-dataset-with-label/GTA_SanAndreasDriving/screenshots"
file_list = os.listdir(ss_path)
keys = ["W","S","A","D"]
keys_list = np.array([list(file_name.split("_")[1].split(".")[0]) for file_name in file_list]).astype(int)

LAod images

In [64]:
image_gen = tf.keras.utils.image_dataset_from_directory(
    ss_path,
    labels=None,
    color_mode='rgb',
    batch_size=16,
    image_size=im_size[:2],
    shuffle=False,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False
)

Found 5709 files belonging to 1 classes.


In [65]:
from tqdm import tqdm
enc_img_list = model.predict(image_gen,steps = len(image_gen))
# for image_path in tqdm(file_list):
#     im = np.array(Image.open(os.path.join(ss_path,image_path)))
#     im = np.expand_dims(im/255,0)
#     conv_im = model.predict(im,verbose = False)
#     enc_img_list.append(conv_im)


357/357 [==============================] - 449s 1s/step


In [66]:
enc_img_list = np.array(enc_img_list).squeeze()
keys_list = np.array(keys_list)

In [115]:
import xgboost as xgb

In [68]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(enc_img_list,keys_list,test_size=.2)

In [116]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.multioutput import MultiOutputClassifier
early_stop = xgb.callback.EarlyStopping(
    rounds=2, metric_name='logloss', data_name='Validation_0', save_best=True
)
# rf = MultiOutputClassifier(HistGradientBoostingClassifier(early_stopping = True, verbose  = True,max_iter = 10000, learning_rate = 0.01,
#                                                          n_iter_no_change = 100, scoring = "f1",min_samples_leaf = 1, 
#                                                           max_leaf_nodes = 14))
rf = MultiOutputClassifier(xgb.XGBClassifier())
rf.fit(x_train,y_train)
print(rf.score(x_test,y_test),
rf.score(x_train,y_train))

0.45796847635726795 0.9991241515217867


0.45096322241681264

0.503831837092183